In [1]:
import os
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ezc3d import c3d
from scipy.signal import resample
from scipy.io import loadmat, savemat
from joblib import load, dump
from tqdm import tqdm

In [1]:
PATH_DATASETS = '../../Dataset/'
PATH_DATASET_1 = 'gait-dbase-1/'

N_SAMPLES = 512

In [3]:
path = os.path.join(PATH_DATASETS, PATH_DATASET_1)
subjects = os.listdir(path)

features = pd.DataFrame()
target = pd.DataFrame()

for subject in tqdm(subjects):
    files_path = os.path.join(path, subject)
    files = os.listdir(files_path)
    for filename in files:

        try:
            content = c3d(os.path.join(path, subject, filename), extract_forceplat_data=True)
            trajectory_x = content['data']['points'][1, 9:15, :]
            trajectory_y = content['data']['points'][1, 9:15, :]
            trajectory_z = content['data']['points'][2, 9:15, :]

            trajectory_x = resample(trajectory_x, N_SAMPLES, axis=1).T
            trajectory_y = resample(trajectory_y, N_SAMPLES, axis=1).T
            trajectory_z = resample(trajectory_z, N_SAMPLES, axis=1).T
            
            _features = pd.DataFrame(
                np.concatenate([trajectory_x, trajectory_y, trajectory_z], axis=1),
                columns=[
                    'lfal_x', 'ltam_x', 'lfcc_x', 'lfm1_x', 'lfm2_x', 'lfm5_x',
                    'lfal_y', 'ltam_y', 'lfcc_y', 'lfm1_y', 'lfm2_y', 'lfm5_y',
                    'lfal_z', 'ltam_z', 'lfcc_z', 'lfm1_z', 'lfm2_z', 'lfm5_z'
                ]
            )

            force_x = content['data']['platform'][0]['force'][0, :]
            force_y = content['data']['platform'][0]['force'][1, :]
            force_z = content['data']['platform'][0]['force'][2, :]

            force_x = resample(force_x, N_SAMPLES, axis=0).T
            force_y = resample(force_y, N_SAMPLES, axis=0).T
            force_z = resample(force_z, N_SAMPLES, axis=0).T

            _target = pd.DataFrame(
                np.stack([force_x, force_y, force_z], axis=1),
                columns=['fx', 'fy', 'fz']
            )

            features = pd.concat([features, _features], axis=0)
            target = pd.concat([target, _target], axis=0)

        except Exception as e:
            # print('Error occured', e)
            pass



100%|██████████| 50/50 [04:03<00:00,  4.88s/it]


In [4]:
features

,lfal_x,ltam_x,lfcc_x,lfm1_x,lfm2_x,lfm5_x,lfal_y,ltam_y,lfcc_y,lfm1_y,lfm2_y,lfm5_y,lfal_z,ltam_z,lfcc_z,lfm1_z,lfm2_z,lfm5_z
0,315.077850,229.390457,266.381287,249.976181,307.238617,358.050293,315.077850,229.390457,266.381287,249.976181,307.238617,358.050293,75.325401,92.450508,39.130699,22.079981,35.831905,21.600080
1,311.619596,225.518568,263.876385,247.135499,304.630233,356.808721,311.619596,225.518568,263.876385,247.135499,304.630233,356.808721,77.493047,92.190563,42.817798,15.211522,30.329800,18.300263
2,313.162473,227.252454,264.989597,248.522765,305.844398,357.182654,313.162473,227.252454,264.989597,248.522765,305.844398,357.182654,77.704683,93.378374,42.899788,18.604518,33.062358,20.060689
3,315.874103,230.292989,266.958884,250.884295,307.941819,357.933013,315.874103,230.292989,266.958884,250.884295,307.941819,357.933013,77.476672,94.913417,42.181133,24.308817,37.642053,22.946695
4,316.499925,230.992649,267.435308,251.468414,308.434996,358.003072,316.499925,230.992649,267.435308,251.468414,308.434996,358.003072,78.149418,95.921182,43.085938,25.705894,38.769354,23.723877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,344.960885,439.361664,397.784377,435.703760,378.716465,331.961474,344.960885,439.361664,397.784377,435.703760,378.716465,331.961474,67.060012,91.399235,8.078960,73.828638,89.778693,59.614009
508,343.137537,437.567519,395.645951,433.907766,377.012505,331.181305,343.137537,437.567519,395.645951,433.907766,377.012505,331.181305,65.591957,89.722308,4.948013,74.723363,90.379700,59.319175
509,346.717951,441.412200,398.912712,439.756271,382.842566,335.393917,346.717951,441.412200,398.912712,439.756271,382.842566,335.393917,64.424899,88.254433,7.105742,66.505498,82.741355,53.307119
510,343.302850,437.967856,395.244875,435.861783,379.069766,333.169987,343.302850,437.967856,395.244875,435.861783,379.069766,333.169987,63.235123,86.861761,3.293285,69.670258,85.333197,54.400806


In [5]:
target

,fx,fy,fz
0,-0.000089,-0.002219,-0.010389
1,0.000179,0.002565,0.014648
2,-0.000280,-0.002959,-0.019489
3,0.000394,0.003411,0.025045
4,-0.000526,-0.003934,-0.031488
...,...,...,...
507,-0.003587,-0.001864,0.063339
508,0.003594,0.001869,-0.063411
509,-0.003600,-0.001873,0.063484
510,0.003607,0.001878,-0.063556


In [6]:
path = '../../Dataset/Processed/data.joblib'
data = {
    'X': features,
    'y': target
}
dump(data, path)

['../../Dataset/Processed/data.joblib']